# Controlling Robot
Introducing basic Proportional Controller

In [1]:
from xiron_py.comms import XironContext
from xiron_py.data import Pose, Twist
import time
import math

In [2]:
ctx = XironContext()
ctx.run_in_separate_thread()

Initialised the main communicator
Signal handler is not added. Make sure to call `stop` function to stop the websockets properly
Connected to S2C WebSocket
Connected to C2S WebSocket
Cleaning up...
Shutting down tasks...
Event loop closed cleanly


In [5]:
def moving_forward_a_given_distance(distance: float, success_threshold: float = 0.05) -> None:
    current_robot_pose = ctx.get_pose("robot0")
    x = current_robot_pose.position[0]
    goal_x = x + distance
    current_delta_to_goal = goal_x - x

    # Control Parameters
    gain = 1.5
    max_vel = 0.5

    print(f"Distance to travel: {goal_x}. Current delta: {current_delta_to_goal}")
    print("Starting Control Loop...")
    while current_delta_to_goal > success_threshold:
        current_robot_pose = ctx.get_pose("robot0")
        x = current_robot_pose.position[0]
        current_delta_to_goal = goal_x - x

        proportional_velocity = gain * current_delta_to_goal
        proporitional_velocity_clamped = min(max_vel, proportional_velocity)

        print(f"Current delta: {current_delta_to_goal}. P Velocity: {proportional_velocity}. P Velocity Clamped: {proporitional_velocity_clamped}")

        # Construct Twist message
        msg = Twist(timestamp=ctx.now(), robot_id="robot0", linear=(proporitional_velocity_clamped, 0.0), angular=0.0)
        ctx.publish_velocity(msg)

        # Control the loop rate
        time.sleep(0.2)

    # Stop the Robot
    msg = Twist(timestamp=ctx.now(), robot_id="robot0", linear=(0.0, 0.0), angular=0.0)
    ctx.publish_velocity(msg)
    print("Completed control loop")

In [6]:
moving_forward_a_given_distance(distance=4.0)

Distance to travel: -3.0544919967651367. Current delta: 4.0
Starting Control Loop...
Current delta: 4.0. P Velocity: 6.0. P Velocity Clamped: 0.5
Current delta: 3.950000762939453. P Velocity: 5.92500114440918. P Velocity Clamped: 0.5
Current delta: 3.8500022888183594. P Velocity: 5.775003433227539. P Velocity Clamped: 0.5
Current delta: 3.7500038146972656. P Velocity: 5.625005722045898. P Velocity Clamped: 0.5
Current delta: 3.641672134399414. P Velocity: 5.462508201599121. P Velocity Clamped: 0.5
Current delta: 3.5416736602783203. P Velocity: 5.3125104904174805. P Velocity Clamped: 0.5
Current delta: 3.4416751861572266. P Velocity: 5.16251277923584. P Velocity Clamped: 0.5
Current delta: 3.341676712036133. P Velocity: 5.012515068054199. P Velocity Clamped: 0.5
Current delta: 3.241678237915039. P Velocity: 4.862517356872559. P Velocity Clamped: 0.5
Current delta: 3.1416797637939453. P Velocity: 4.712519645690918. P Velocity Clamped: 0.5
Current delta: 3.0416812896728516. P Velocity: 4.

In [7]:
# Reset simulation
ctx.reset_simulation()

# What about rotation

In [11]:
def rotating_a_give_angle(angle: float, success_threshold: float = 0.05) -> None:
    current_robot_pose = ctx.get_pose("robot0")
    theta = current_robot_pose.orientation # always between -pi to pi
    goal_theta = theta + angle
    current_delta_to_goal = goal_theta - theta

    # Control Parameters
    gain = 1.5
    max_vel = 0.5

    print(f"Distance to travel: {goal_theta}. Current delta: {current_delta_to_goal}")
    print("Starting Control Loop...")
    while current_delta_to_goal > success_threshold:
        current_robot_pose = ctx.get_pose("robot0")
        theta = current_robot_pose.orientation
        current_delta_to_goal = goal_theta - theta

        proportional_velocity = gain * current_delta_to_goal
        proporitional_velocity_clamped = min(max_vel, proportional_velocity)

        print(f"Current delta: {current_delta_to_goal}. P Velocity: {proportional_velocity}. P Velocity Clamped: {proporitional_velocity_clamped}")

        # Construct Twist message
        msg = Twist(timestamp=ctx.now(), robot_id="robot0", linear=(0.0, 0.0), angular=proporitional_velocity_clamped)
        ctx.publish_velocity(msg)

        # Control the loop rate
        time.sleep(0.2)

    # Stop the Robot
    msg = Twist(timestamp=ctx.now(), robot_id="robot0", linear=(0.0, 0.0), angular=0.0)
    ctx.publish_velocity(msg)
    print("Completed control loop")

In [12]:
rotating_a_give_angle(math.pi/2)

Distance to travel: 1.5707963267948966. Current delta: 1.5707963267948966
Starting Control Loop...
Current delta: 1.5707963267948966. P Velocity: 2.356194490192345. P Velocity Clamped: 0.5
Current delta: 1.5041296566512923. P Velocity: 2.2561944849769384. P Velocity Clamped: 0.5
Current delta: 1.4041296253588538. P Velocity: 2.1061944380382807. P Velocity Clamped: 0.5
Current delta: 1.3041295866158347. P Velocity: 1.956194379923752. P Velocity Clamped: 0.5
Current delta: 1.2041296819832663. P Velocity: 1.8061945229748995. P Velocity Clamped: 0.5
Current delta: 1.104129777350698. P Velocity: 1.656194666026047. P Velocity Clamped: 0.5
Current delta: 1.0041298429158072. P Velocity: 1.5061947643737108. P Velocity Clamped: 0.5
Current delta: 0.9041299382832388. P Velocity: 1.3561949074248583. P Velocity Clamped: 0.5
Current delta: 0.8041300336506705. P Velocity: 1.2061950504760057. P Velocity Clamped: 0.5
Current delta: 0.7041301290181021. P Velocity: 1.0561951935271532. P Velocity Clamped:

In [13]:
# Reset simulation. This should bring you back to original state
ctx.reset_simulation()

In [14]:
# Try running the control with various. It should fail for some of the tasks

In [15]:
# Lets use a better Controller for this

In [16]:
from xiron_py.controller.pid import PIDController, PIDConfig
import numpy as np

In [17]:
pid_controller = PIDController()
pid_controller.distance_threshold = 0.05

Initialised PID Controller


In [18]:
plan = [[0.0, -3.0, 0.0], [0.0, 0.0, 0.0]]
pid_controller.set_plan(plan)

In [19]:
# Lets try the control loop again
final_goal_vec = np.array(plan[-1]).reshape(-1, 1) # Shape becomes (3, 1)
current_pose = ctx.get_pose("robot0")
current_pose_vec = np.array([current_pose.position[0], current_pose.position[1], current_pose.orientation]).reshape(-1, 1) # (3, 1)

distance = pid_controller.distance(final_goal_vec, current_pose_vec)
last_control = np.zeros((2, 1))

print(f"Distance to final point: {distance}. Distance threshold: {pid_controller.distance_threshold}")
while distance > pid_controller.distance_threshold:
    current_pose = ctx.get_pose("robot0")
    current_pose_vec = np.array([current_pose.position[0], current_pose.position[1], current_pose.orientation]).reshape(-1, 1) # (3, 1)
    print(f"Distance to final point: {distance}. Distance threshold: {pid_controller.distance_threshold}")
    distance = pid_controller.distance(final_goal_vec, current_pose_vec)
    
    vel = pid_controller.compute_contol(current_pose_vec, last_control)

    msg = Twist(timestamp=ctx.now(), robot_id="robot0", linear=(vel[0][0], 0.0), angular=vel[1][0])
    ctx.publish_velocity(msg)

    # Set variable
    last_control = vel

    time.sleep(0.2)

print("Reached goal position. Setting zero velocity")
msg = Twist(timestamp=ctx.now(), robot_id="robot0", linear=(0.0, 0.0), angular=0.0)
ctx.publish_velocity(msg)

Distance to final point: 7.637299352975271. Distance threshold: 0.05
Distance to final point: 7.637299352975271. Distance threshold: 0.05
Distance to final point: 7.637299352975271. Distance threshold: 0.05
Distance to final point: 7.552710780250658. Distance threshold: 0.05
Distance to final point: 7.460623012057221. Distance threshold: 0.05
Distance to final point: 7.368741460766153. Distance threshold: 0.05
Distance to final point: 7.277073937568832. Distance threshold: 0.05
Distance to final point: 7.185628633580091. Distance threshold: 0.05
Distance to final point: 7.094414141862718. Distance threshold: 0.05
Distance to final point: 7.003439480887262. Distance threshold: 0.05
Distance to final point: 6.912714119525865. Distance threshold: 0.05
Distance to final point: 6.86744801489848. Distance threshold: 0.05
Distance to final point: 6.7771154083175045. Distance threshold: 0.05
Distance to final point: 6.687057923154927. Distance threshold: 0.05
Distance to final point: 6.5972868

In [20]:
ctx.reset_simulation()

In [ ]:
# Feel free to modify the controller and play around

In [22]:
ctx.stop()